In [2]:
from __future__ import print_function
import os
import cv2 as cv
import numpy as np
import random as rng
from PIL import Image as PILImage
myShiTomasi_window = 'window'
data_path = '/media/data1/image_data/chess/'
patch_width = 50
rng.seed(12345)

In [3]:
def make_3d(gray_img):
    # indexing with np.newaxis inserts a new 3rd dimension, which we then repeat the
    # array along, (you can achieve the same effect by indexing with None)
    return np.repeat(gray_img[:, :, np.newaxis], 3, axis=2)

def show_corners(corners, src_img):
    src_copy = np.copy(src_img)
    cv.namedWindow('corners')
    for i,j in corners:
        cv.circle(src_copy, (round(j),round(i)), 4,  (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256)), cv.FILLED)
    cv.imshow('corners', src_copy)
    cv.waitKey()
    cv.destroyWindow('corners')
                
    
def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 0)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value

In [38]:
# Load source image and convert it to gray
# img_filename = 'board_with_pieces.jpeg' # done
# img_filename = 'board_with_pieces_overhead.png' # done
# img_filename = 'IMG_0645.JPG' # done
# img_filename = 'IMG_0646.JPG'
# img_filename = 'IMG_0654.JPG' # done
# img_filename = 'empty_chessboard.png' # done
# img_filename = 'IMG_0650.JPG' # done
# img_filename = 'board_wood_grain.jpg' # done
# img_filename = 'board_wood_grain_oh.jpeg' # done
# img_filename = 'board_wood_grain_with_pieces.jpg' # done
# img_filename = 'board_wood_grain_red.jpg' # done
index_dot = img_filename.index('.')
img_filename_no_ext = img_filename[:index_dot] if index_dot > -1 else img_filename
src = cv.imread(f'{data_path}{img_filename}')
assert src is not None
print(src.shape)
# src_small = src
src_gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
src_small = cv.resize(src, None, fx=0.5, fy=0.5)
src_gray = cv.resize(src_gray, None, fx=0.5, fy=0.5)
print(src_gray.shape)
padded_src_gray = np.pad(src_gray, patch_width, pad_with)

(1200, 1600, 3)
(600, 800)


In [39]:
src_gray = cv.equalizeHist(src_gray)
show_corners([], src_gray)

In [35]:
# Set some parameters
blockSize = 5
min_distance = 10
quality_level = .2

In [40]:
# experimenting with goodFeaturesToTrack (it will do clustering for me)
corners = cv.goodFeaturesToTrack(src_gray, 256, quality_level, min_distance)
corner_coords = corners[:, 0, ::-1]
show_corners(corner_coords, src_small)
pc_list = corner_coords.tolist()
rng.shuffle(pc_list)

In [41]:
# USE THIS TO GENERATE SAMPLE DATA FOR ML
qKey = 113
backKey = 8
enterKey = 13
cv.namedWindow('a')
pos_count = 0
neg_count = 0
for i, j in pc_list:
    i = round(i)
    j = round(j)
    i += patch_width # to get out of the 0 padding
    j += patch_width # to get out of the 0 padding
    clean_corner_image = PILImage.fromarray(padded_src_gray[i-patch_width:i+patch_width, j-patch_width:j+patch_width])
    corner_image = np.copy(clean_corner_image)
    cv.circle(corner_image, (patch_width,patch_width), 4, (0, 0, 0), cv.FILLED)
    if pos_count < 49:
        cv.imshow('a', corner_image)
        cv.moveWindow('a', 2800, 500)
        b = cv.waitKey()
    else:
        b = backKey
    if b == qKey:
        break
    if b == backKey:
        neg_count += 1
        label = 0
        count = neg_count
    else:
        pos_count += 1
        label = 1
        count = pos_count
    clean_corner_image.save(f'{data_path}corner_data/{img_filename_no_ext}_{patch_width}_{label}_{count:04d}.jpeg')
    print(f'{neg_count + pos_count} / {len(pc_list)}')
cv.destroyAllWindows()

1 / 85
2 / 85
3 / 85
4 / 85
5 / 85
6 / 85
7 / 85
8 / 85
9 / 85
10 / 85
11 / 85
12 / 85
13 / 85
14 / 85
15 / 85
16 / 85
17 / 85
18 / 85
19 / 85
20 / 85
21 / 85
22 / 85
23 / 85
24 / 85
25 / 85
26 / 85
27 / 85
28 / 85
29 / 85
30 / 85
31 / 85
32 / 85
33 / 85
34 / 85
35 / 85
36 / 85
37 / 85
38 / 85
39 / 85
40 / 85
41 / 85
42 / 85
43 / 85
44 / 85
45 / 85
46 / 85
47 / 85
48 / 85
49 / 85
50 / 85
51 / 85
52 / 85
53 / 85
54 / 85
55 / 85
56 / 85
57 / 85
58 / 85
59 / 85
60 / 85
61 / 85
62 / 85
63 / 85
64 / 85
65 / 85
66 / 85
67 / 85
68 / 85
69 / 85
70 / 85
71 / 85
72 / 85
73 / 85
74 / 85
75 / 85
76 / 85
77 / 85
78 / 85
79 / 85
80 / 85
81 / 85
82 / 85
83 / 85
84 / 85
85 / 85


In [12]:
cv.destroyAllWindows()